# [실기형 1번] TensorFlow를 이용한 MNIST 분류 모델 구현

문제에 주어진 그림의 아키텍처를 반영하는 TensorFlow 모델을 구현하시오

---


※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [ ]:
from tensorflow.keras.layers import MaxPool2D, Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
?Dense

In [ ]:
model1 = Sequential([
##### 여기에 코드를 작성하세요 #####
    # filters = 10, kernel_size = (5,5), Input 28x28 gray 
    Conv2D(10,5,input_shape=(28,28,1)), # 24 = (28-5)/1 + 1 
    MaxPool2D(pool_size=(2,2)), # 12 = 24 / 2
    Conv2D(20,5), # 8 = (12-5)/1 + 1
    MaxPool2D(pool_size=(2,2)), # 4 = 8 /2
    Flatten(), # 4x4x20 -> 
    Dropout(0.5), # 
    Dense(100), # 
    Dense(10)
#####################################
])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 24, 24, 10)        260       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 20)          5020      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 20)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 320)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 320)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)              

# [실기형 2번] TensorFlow 모델 코드 수정 및 학습하기

학습이 정상적으로 이루어질 수 있도록 주어진 코드를 수정하고, 학습하여 결과(Accuray)가 0.9 이상이 나오도록 하시오. 

---


※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten 

tf.random.set_seed(100)
 
(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_x = train_x[:10000]
test_x = test_x[:10000]
train_y = train_y[:10000]
test_y = test_y[:10000]


In [ ]:
# Conv2D 입력을 위해 (28,28) -> (28,28,1) 로 shape변경
train_x = train_x.reshape((train_x.shape[0], 28, 28, 1))
test_x = test_x.reshape((test_x.shape[0], 28, 28, 1))

In [ ]:
# one hot encoding of labels
# train_y = to_categorical(train_y)
# test_y = to_categorical(test_y)

In [ ]:
# 데이터 타입이 uint8이고, 0~ 255 사이의 값이다. 그대로 사용가능한가?
train_x[0].ravel()[200:210]

array([  0,   0,   0,  49, 238, 253, 253, 253, 253, 253], dtype=uint8)

In [ ]:
# 정규화를 해볼꺼나.. 안하면? 성능이 개판이다.
train_norm = train_x / 255.0
test_norm = test_x / 255.0

In [ ]:
# 모델2 설계를 잘해 보자. 여러번 테스트 하려면 처음부터 다시 해야 함.
model2 = Sequential()
model2.add(Conv2D(16, (3, 3), 
                  activation='relu', 
                  kernel_initializer='glorot_normal', 
                  input_shape=(28, 28, 1)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(15, activation='relu', kernel_initializer='glorot_normal'))
model2.add(Dense(10, activation='softmax'))
# 옵티마이저를 잘 설정해보자. 러닝레이트와 모멘텀을 잘 이용해 보자.
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
# 손실함수는 사실 바꿀 이유가 없다.
model2.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 모델 훈련!!!
history = model2.fit(train_norm,train_y, epochs=5, batch_size=32, verbose=2)

Epoch 1/5
313/313 - 3s - loss: 0.6301 - accuracy: 0.8029
Epoch 2/5
313/313 - 3s - loss: 0.2986 - accuracy: 0.9184
Epoch 3/5
313/313 - 3s - loss: 0.2039 - accuracy: 0.9407
Epoch 4/5
313/313 - 3s - loss: 0.2249 - accuracy: 0.9391
Epoch 5/5
313/313 - 3s - loss: 0.2557 - accuracy: 0.9304


In [ ]:
history.history

{'accuracy': [0.8029000163078308,
  0.91839998960495,
  0.9406999945640564,
  0.9391000270843506,
  0.930400013923645],
 'loss': [0.630146861076355,
  0.298598974943161,
  0.20386672019958496,
  0.22487638890743256,
  0.2556667625904083]}

In [ ]:
print('\n=== Result for the test dataset is as follows ===')
pred = model2.evaluate(test_norm, test_y,verbose=1) 


=== Result for the test dataset is as follows ===
313/313 [==============================] - 2s 5ms/step - loss: 0.3780 - accuracy: 0.9086


In [ ]:
pred

[0.37800657749176025, 0.9085999727249146]

# [실기형 3번] Learning rate 조절하기

주어진 조건에 따라 코드를 작성하시오

---

※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [ ]:
import tensorflow as tf

''' MNIST data setting'''
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10) 

''' Feedforward Model setting'''
model3 = tf.keras.models.Sequential([
###### 여기에 코드를 작성하시오 ######
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
######################################
])

''' Model build '''
###### 여기에 코드를 작성하시오 ######
def lr_schedule(epoch):
  if epoch < 10:
    lr = 0.1 + (epoch * 0.1) 
  else:
    lr = 1.0 - (epoch * 0.04)
  return lr

callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)
model3.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
######################################
history = model3.fit(x_train, y_train, epochs=25, validation_data=(x_test, y_test), callbacks=[callback], verbose=1)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/25

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3236 - accuracy: 0.9071 - val_loss: 0.1883 - val_accuracy: 0.9434
Epoch 2/25

Epoch 00002: LearningRateScheduler reducing learning rate to 0.2.
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1778 - accuracy: 0.9472 - val_loss: 0.1361 - val_accuracy: 0.9590
Epoch 3/25

Epoch 00003: LearningRateScheduler reducing learning rate to 0.30000000000000004.
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1372 - accuracy: 0.9583 - val_loss: 0.1282 - val_accuracy: 0.9627
Epoch 4/25

Epoch 00004: LearningRateScheduler reducing learning rate to 0.4.
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1236 - accuracy: 0.9620 - val_loss: 0.1291 - val_accuracy: 0.9617
Epoch 5/25

Epoch 00005: LearningRateScheduler reducing learning rate to 0.

# [실기형 4번] CNN(Convolutional Neural Network) 모델 변경하기

[문제4-1] 문제에 주어진 조건에 따라 코드를 작성하시오.

---


※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd '/content/drive/MyDrive/Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [5]:
ls -al

total 30449
-rw------- 1 root root    46360 Jul  4 12:24 AI_BASIC_V2020.ipynb
-rw------- 1 root root 31035704 Jul  4 08:01 data_batch_1
-rw------- 1 root root    96528 Jun 29 03:11 fashionmnist_v2_Affine.ipynb


In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

tf.random.set_seed(2020)

''' CIFAR-10 dataset setting'''
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding = 'bytes')
        return dict

train1 = unpickle('./data_batch_1')

In [7]:
train1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [8]:
# type(train1[b'data'])
train1[b'data'].shape

(10000, 3072)

In [9]:
# CNN을 적용하기위해 (10000, 3072) -> (10000,32,32,3) 으로 변경 
x_train = train1[b'data'].reshape(10000,32,32,3)
y_train = train1[b'labels'] # 리스트 타입이다.
y_train = np.array(y_train, dtype = 'uint8') # ndarray로 변경

In [10]:
print(y_train.shape)
print(np.expand_dims(y_train, axis=1).shape)
print(y_train.reshape(-1,1).shape)

(10000,)
(10000, 1)
(10000, 1)


In [11]:
y_train = np.expand_dims(y_train, axis=1)  # y_train에 1 디멘젼 추가
x_train = x_train / 255.0  # x_train 정규화 

In [12]:
# 모델 작성 : Functional API 방식 
inputs = tf.keras.Input(shape=(32, 32, 3)) # Input 클래스 선언 
x = inputs
##### 여기에 코드를 작성하세요 #####
x = layers.Conv2D(16, 5, activation='relu', padding='same')(x) # padding size = 2 의 의미는 same 옵션임. 
x = layers.MaxPooling2D(2,2)(x) # pooling size = 2, stride = 2
x = layers.Conv2D(16, 5, activation='relu', padding='same')(x) 
x = layers.Flatten()(x) 
x = layers.Dense(16)(x) 
x = layers.Dense(10, activation='softmax')(x)
#####################################
outputs = x
model4_1 = tf.keras.Model(inputs, outputs)
model4_1.summary()  

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 16)        6416      
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                65552     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                170   

In [13]:
model4_1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model4_1.fit(x_train[:5000],y_train[:5000],epochs=5)

Epoch 1/5
157/157 [==============================] - 32s 3ms/step - loss: 2.0280 - accuracy: 0.2616
Epoch 2/5
157/157 [==============================] - 0s 3ms/step - loss: 1.7039 - accuracy: 0.3948
Epoch 3/5
157/157 [==============================] - 0s 3ms/step - loss: 1.5238 - accuracy: 0.4570
Epoch 4/5
157/157 [==============================] - 0s 3ms/step - loss: 1.4033 - accuracy: 0.5068
Epoch 5/5
157/157 [==============================] - 0s 3ms/step - loss: 1.3010 - accuracy: 0.5432



[문제4-2] 문제에 주어진 조건에 따라 코드를 작성하시오.

---


※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [24]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs
##### 여기에 코드를 작성하세요 #####
x = inputs
x = layers.Conv2D(32, 3, activation='relu', padding="valid")(x)
x = layers.MaxPooling2D(2,2)(x)
x = layers.Conv2D(12, 3, activation='relu', padding="valid")(x)
x = layers.Flatten()(x)
x = layers.Dense(16)(x) 
x = layers.Dense(10, activation='softmax')(x)
#####################################
outputs = x
model4_2 = tf.keras.Model(inputs, outputs)
model4_2.summary()  

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 12)        3468      
_________________________________________________________________
flatten_9 (Flatten)          (None, 2028)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 16)                32464     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                170 

In [19]:
model4_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model4_2.fit(x_train[:5000], y_train[:5000], epochs=5)

Epoch 1/5
157/157 [==============================] - 1s 3ms/step - loss: 2.0737 - accuracy: 0.2526
Epoch 2/5
157/157 [==============================] - 0s 3ms/step - loss: 1.7434 - accuracy: 0.3888
Epoch 3/5
157/157 [==============================] - 0s 3ms/step - loss: 1.5749 - accuracy: 0.4470
Epoch 4/5
157/157 [==============================] - 0s 3ms/step - loss: 1.4761 - accuracy: 0.4812
Epoch 5/5
157/157 [==============================] - 0s 3ms/step - loss: 1.3932 - accuracy: 0.5080


# [실기형 5번] TensorFlow 모델 구현하기

문제에 주어진 [그림.1]의 subnetwork와 [그림.2]의 모델에 대한 코드를 작성하시오.

---


※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [26]:
import tensorflow as tf
from tensorflow.keras import layers 

def subnetwork(x):
##### 여기에 코드를 작성하세요 (4점) #####
    x_1 = layers.Conv2D(filters = 5, kernel_size = 3, padding='same')(x)
    x_2 = layers.Conv2D(filters = 5, kernel_size = 3, padding='same')(x)
    x_2 = layers.Conv2D(filters = 5, kernel_size = 3, padding='same')(x_2)
    x_3 = layers.MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x) # pooling_size (3,3), 크기는 유지한다. 
    return layers.concatenate([x_1, x_2, x_3], axis=3)
#########################################


inputs = tf.keras.Input(shape=(32, 32, 3))
x = inputs

##### 여기에 코드를 작성하세요 (3점) #####
x = subnetwork(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x) # 1/2 로 줄인다. 
x = subnetwork(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)
x = subnetwork(x)
x = layers.AveragePooling2D(pool_size=(8,8), strides=2)(x) # 입력이 (8,8,33) 이다. 출력을 (1,1,33) 으로 한다. 
x = layers.Dense(5, activation='softmax')(x)
#########################################
outputs = x
model5 = tf.keras.Model(inputs, outputs)
model5.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, 32, 32, 5)    140         input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 32, 32, 5)    140         input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 32, 32, 5)    230         conv2d_30[0][0]                  
___________________________________________________________________________________________

# [실기형 6번] 민원 자동 분류

[문제6-1] 주어진 조건에 따라 코드를 작성하시오

---

※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [27]:
import tensorflow as tf

model_rnn_1 = tf.keras.Sequential([
    ##### 이 부분에 코드를 작성하시오 #####           
    tf.keras.layers.Embedding(1000002, 32), # 100만개 단어, [PAD]와 [OOV] 추가 
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(6, activation='softmax')    
    #######################################
])  

model_rnn_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          32000064  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 306       
Total params: 32,016,970
Trainable params: 32,016,970
Non-trainable params: 0
_________________________________________________________________




[문제6-2] 주어진 조건에 따라 코드를 작성하시오

> 참고) 민원 예시 문장을 vocab의 index로 변환하면 다음과 같다
* minwon = [['유치원','앞', '신호등이', '고장', '났어요', '.']]
* minwon_ids = [[100, 43, 75623, 787, 345, 16]]

---

※ 작성 대상 영역 외에는 수정하지 않도록 합니다.

In [30]:
''' 모델 개선하기 '''
model_rnn_2 = tf.keras.Sequential([
    ##### 이 부분에 코드를 작성하시오 #####           
    tf.keras.layers.Embedding(1000002, 32),
    tf.keras.layers.LSTM(50, return_sequences=True), # LSTM을 한층 더 쌓는다. 
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(6, activation='softmax')
    #######################################
])  

model_rnn_2.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          32000064  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 50)          16600     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 306       
Total params: 32,037,170
Trainable params: 32,037,170
Non-trainable params: 0
_________________________________________________________________


In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
''' 
민원 문장 자르기 처리하기
'''
minwon_ids = [[123, 124, 346, 76, 66, 1221, 8762, 4574, 66, 1, 7, 999, 2, 4, 98685, 52, 10, 40, 124, 127, 789654, 27, 122, 9, 1991, 4, 8, 15, 16, 23, 42]] 
minwon_ids_result = pad_sequences(minwon_ids, maxlen=25, truncating='post') # 문장의 뒷부분 잘라내기 

print(minwon_ids_result)

[[   123    124    346     76     66   1221   8762   4574     66      1
       7    999      2      4  98685     52     10     40    124    127
  789654     27    122      9   1991]]
